# CPS5

### Setting up the environment 

In [58]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thenvironmentu Sep 27 22:49:33 2018

@author: yassine
"""

################################################
######## Importing useful modules ##############
################################################

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import scipy as sp
from numba import jit
import matplotlib.pyplot as plt
import scipy.stats as sps

%matplotlib qt5


   

### Question 1

In [60]:
%%capture 
## the previous line is to supress unecessary outputs


####### Classic MonteCarlo estimation ######
@jit
def classic_MonteCarlo(S0,r,sigma,K,T,N):
    
    BT = np.random.normal(0,np.sqrt(T),M)
    ST = S0*np.exp( (r-(sigma**2)/2) + sigma*BT)
    
    return np.mean(ST>=K)



### Question 2

In [63]:
%%capture 
## the previous line is to supress unecessary outputs



####### Importance Sampling MonteCarlo######
@jit
def changeOfMeasure_MonteCarlo(S0,r,sigma,K,T,M,theta):
    
    # Brownian motion under distribution Q
    BT_forQ = np.random.normal(0,np.sqrt(T),M) + theta*T
    
    # ST simulated under distribution Q
    ST_forQ = S0*np.exp( (r-(sigma**2)/2) + sigma*BT_forQ)
    
    # Likelihood or Q's density with respect to P 
    Likelihood = np.exp(theta*BT_forQ - 0.5*(theta**2)*T)
    
    return np.mean((ST_forQ>=K)/Likelihood)


### Question 3

In [93]:
###############################################
####### Defining parameters and constants######
###############################################

def exactValue(S0,r,sigma,K,T):
    
    """ This function computes the exact value of the probability we are estimating """
    return sps.norm.cdf( (- np.log(K/S0) + (r-sigma**2)/2 ) / sigma )


T  = 1
S0 = 100
K  = 150
r  = 0.02
sigma = 0.4
M  = 1000





theta_list = np.linspace(-3,3,101)

estimations_errors = np.array([changeOfMeasure_MonteCarlo(S0,r,sigma,K,T,M,N,theta) for theta in theta_list ]) - exactValue(S0,r,sigma,K,T)

plt.figure("Figure 1")
plt.plot(theta_list,estimations_errors,label="estimation errors for different theta values")
plt.grid(True)


### Question 4

In [101]:
####### Importance Sampling MonteCarlo######
@jit
def Variance_estimator(S0,r,sigma,K,T,M,theta):
    
    # Brownian motion under distribution Q
    BT_forQ = np.random.normal(0,np.sqrt(T),M) + theta*T
    
    # ST simulated under distribution Q
    ST_forQ = S0*np.exp( (r-(sigma**2)/2) + sigma*BT_forQ)
    
    # Likelihood or Q's density with respect to P 
    Likelihood = np.exp(theta*BT_forQ - 0.5*(theta**2)*T)
    
    Delta0 = exactValue(S0,r,sigma,K,T)
    
    return np.mean((ST_forQ>=K)/(Likelihood**2)) - (Delta0**2)


In [129]:
T  = 1
S0 = 100
K  = 150
r  = 0.02
sigma = 0.4
M  = 100000




theta_list = np.linspace(-1,3,1000)

Variance_estimations = [Variance_estimator(S0,r,sigma,K,T,M,N,theta) for theta in theta_list ]

plt.figure("Figure 4")

plt.plot(theta_list,Variance_estimations)
